In [ ]:
import cv2 
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

Initiate Model

# SegNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from torch.autograd import Variable

class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.UpsamplingBilinear2d(scale_factor=2)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x


class ConvLSTMCell(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_size: (int, int)
            Height and width of input tensor as (height, width).
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.height, self.width = input_size
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)

        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size):
        return (torch.zeros(batch_size, self.hidden_dim, self.height, self.width).cuda(),
                torch.zeros(batch_size, self.hidden_dim, self.height, self.width).cuda())


class ConvLSTM(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.height, self.width = input_size

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_size=(self.height, self.width),
                                          input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor=input_tensor.permute(1, 0, 2, 3, 4)

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=input_tensor.size(0))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])


                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output = layer_output.permute(1, 0, 2, 3, 4)

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param
  

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

class SegNet_ConvLSTM(nn.Module):
    def __init__(self):
        super(SegNet_ConvLSTM,self).__init__()
        self.vgg16_bn = models.vgg16_bn(pretrained=True).features
        self.relu = nn.ReLU(inplace=True)
        self.index_MaxPool = nn.MaxPool2d(kernel_size=2, stride=2,return_indices=True)
        self.index_UnPool = nn.MaxUnpool2d(kernel_size=2, stride=2)
        # net struct
        self.conv1_block = nn.Sequential(self.vgg16_bn[0],  # conv2d(3,64,(3,3))
                                         self.vgg16_bn[1],  # bn(64,eps=1e-05,momentum=0.1,affine=True)
                                         self.vgg16_bn[2],  # relu(in_place)
                                         self.vgg16_bn[3],  # conv2d(3,64,(3,3))
                                         self.vgg16_bn[4],  # bn(64,eps=1e-05,momentum=0.1,affine=True)
                                         self.vgg16_bn[5]   # relu(in_place)
                                         )
        self.conv2_block = nn.Sequential(self.vgg16_bn[7],
                                         self.vgg16_bn[8],
                                         self.vgg16_bn[9],
                                         self.vgg16_bn[10],
                                         self.vgg16_bn[11],
                                         self.vgg16_bn[12]
                                         )
        self.conv3_block = nn.Sequential(self.vgg16_bn[14],
                                         self.vgg16_bn[15],
                                         self.vgg16_bn[16],
                                         self.vgg16_bn[17],
                                         self.vgg16_bn[18],
                                         self.vgg16_bn[19],
                                         self.vgg16_bn[20],
                                         self.vgg16_bn[21],
                                         self.vgg16_bn[22]
                                         )
        self.conv4_block = nn.Sequential(self.vgg16_bn[24],
                                         self.vgg16_bn[25],
                                         self.vgg16_bn[26],
                                         self.vgg16_bn[27],
                                         self.vgg16_bn[28],
                                         self.vgg16_bn[29],
                                         self.vgg16_bn[30],
                                         self.vgg16_bn[31],
                                         self.vgg16_bn[32]
                                         )
        self.conv5_block = nn.Sequential(self.vgg16_bn[34],
                                         self.vgg16_bn[35],
                                         self.vgg16_bn[36],
                                         self.vgg16_bn[37],
                                         self.vgg16_bn[38],
                                         self.vgg16_bn[39],
                                         self.vgg16_bn[40],
                                         self.vgg16_bn[41],
                                         self.vgg16_bn[42]
                                         )

        self.upconv5_block = nn.Sequential(
                                           nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1, 1)),
                                           nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           )
        self.upconv4_block = nn.Sequential(
                                           nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(512, 256, kernel_size=(3, 3), padding=(1, 1)),
                                           nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           )
        self.upconv3_block = nn.Sequential(
                                           nn.Conv2d(256, 256, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(256, 256, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(256, 128, kernel_size=(3, 3), padding=(1, 1)),
                                           nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           )
        self.upconv2_block = nn.Sequential(
                                           nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(128, 64, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu
                                           )
        self.upconv1_block = nn.Sequential(
                                           nn.Conv2d(64, 64, kernel_size=(3, 3), padding=(1,1)),
                                           nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
                                           self.relu,
                                           nn.Conv2d(64, 2, kernel_size=(3, 3), padding=(1,1)),
                                           )
        self.convlstm = ConvLSTM(input_size=(4,8),
                                 input_dim=512,
                                 hidden_dim=[512, 512],
                                 kernel_size=(3,3),
                                 num_layers=2,
                                 batch_first=False,
                                 bias=True,
                                 return_all_layers=False)
    def forward(self, x):
        x = torch.unbind(x, dim=1)
        data = []
        for item in x:
            f1, idx1 = self.index_MaxPool(self.conv1_block(item))
            f2, idx2 = self.index_MaxPool(self.conv2_block(f1))
            f3, idx3 = self.index_MaxPool(self.conv3_block(f2))
            f4, idx4 = self.index_MaxPool(self.conv4_block(f3))
            f5, idx5 = self.index_MaxPool(self.conv5_block(f4))
            data.append(f5.unsqueeze(0))
        data = torch.cat(data, dim=0)
        lstm, _ = self.convlstm(data)
        test = lstm[0][-1,:,:,:,:]
        up6 = self.index_UnPool(test,idx5)
        up5 = self.index_UnPool(self.upconv5_block(up6), idx4)
        up4 = self.index_UnPool(self.upconv4_block(up5), idx3)
        up3 = self.index_UnPool(self.upconv3_block(up4), idx2)
        up2 = self.index_UnPool(self.upconv2_block(up3), idx1)
        up1 = self.upconv1_block(up2)
        return F.log_softmax(up1, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = '/content/drive/MyDrive/Colab Notebooks/ModelNN/weights_best.pth'
model = SegNet().to(device)
model.load_state_dict(torch.load(path))

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

<All keys matched successfully>

# UNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from torch.autograd import Variable

class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.UpsamplingBilinear2d(scale_factor=2)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x


class ConvLSTMCell(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_size: (int, int)
            Height and width of input tensor as (height, width).
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.height, self.width = input_size
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)

        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size):
        return (torch.zeros(batch_size, self.hidden_dim, self.height, self.width).cuda(),
                torch.zeros(batch_size, self.hidden_dim, self.height, self.width).cuda())


class ConvLSTM(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.height, self.width = input_size

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_size=(self.height, self.width),
                                          input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor=input_tensor.permute(1, 0, 2, 3, 4)

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=input_tensor.size(0))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])


                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output = layer_output.permute(1, 0, 2, 3, 4)

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [ ]:
class UNet_ConvLSTM(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet_ConvLSTM, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)
        self.convlstm = ConvLSTM(input_size=(8,16),
                                 input_dim=512,
                                 hidden_dim=[512, 512],
                                 kernel_size=(3,3),
                                 num_layers=2,
                                 batch_first=False,
                                 bias=True,
                                 return_all_layers=False)

    def forward(self, x):
        x = torch.unbind(x, dim=1)
        data = []
        for item in x:
            x1 = self.inc(item)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)
            x5 = self.down4(x4)
            data.append(x5.unsqueeze(0))
        data = torch.cat(data, dim=0)
        lstm, _ = self.convlstm(data)
        test = lstm[0][ -1,:, :, :, :]
        x = self.up1(test, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x, test

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = '/content/drive/MyDrive/TA_UNetLSTM/weights_best.pth'
model = UNet_ConvLSTM(3,2).to(device)
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = '/content/drive/MyDrive/Colab Notebooks/ModelNN/UNet/weights_best.pth'
model = UNet(3,2).to(device)
model.load_state_dict(torch.load(path))

NameError: ignored

# Listing file

In [ ]:
import os
def getListOfFiles(dirName):
  listOfFile = os.listdir(dirName)
  allFiles = list()
  for item in listOfFile:
    fullPath = os.path.join(dirName,item)
    if os.path.isdir(fullPath):
      allFiles = allFiles + getListOfFiles(fullPath)
    else:
      allFiles.append(fullPath)
  return allFiles

def getList(dirName):
    
    
    
    # Get the list of all files in directory tree at given path
    listOfFiles = getListOfFiles(dirName)
    
    # Print the files
    #for elem in listOfFiles:
        #print(elem)
    #print ("****************")
    
    # Get the list of all files in directory tree at given path
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
        
        
    # Print the files    
    #for elem in listOfFiles:
        #print(elem)  
    return listOfFiles  

# Extraxting Frame and Inference

## cnn-rnn

In [ ]:
import cv2 
import os
from torchvision import transforms
import torchvision
from PIL import Image
convert_tensor = transforms.ToTensor()
cap = cv2.VideoCapture('/content/drive/MyDrive/Video Processing/cerahlurus.mp4')
try: 
  if not os.path.exists('/content/drive/MyDrive/Video Processing/cerahlurusTA_2'):
    os.makedirs('/content/drive/MyDrive/Video Processing/cerahlurusTA_2')
except OSError: 
  print('Error: Creating directory of data')

currentframe = 1
while(True):
  success,frame = cap.read()
  if success:
    path = '/content/drive/MyDrive/Video Processing/cerahlurusTA_2/'
    name = path + 'frame' + str(currentframe) +'.jpg'
    pred_name = path + 'pred' + str(currentframe) + '.jpg'
    print('creating...'+name)

    with torch.no_grad():
      if currentframe % 5 == 0: 
        model.eval()
        frame = cv2.resize(frame,(256,128))
        cv2.imwrite(name, frame)
        list_image = getList(path)
        #print(list_image)
        data = []
        data1 = torch.unsqueeze(convert_tensor(Image.open(list_image[-5])),0)
        data.append(data1)
        data2 = torch.unsqueeze(convert_tensor(Image.open(list_image[-4])),0)
        data.append(data2)
        data3 = torch.unsqueeze(convert_tensor(Image.open(list_image[-3])),0)
        data.append(data3)
        data4 = torch.unsqueeze(convert_tensor(Image.open(list_image[-2])),0)
        data.append(data4)
        data5 = torch.unsqueeze(convert_tensor(Image.open(list_image[-1])),0)
        data.append(data5)
        
        data = torch.cat(data,0).unsqueeze(0).to(device)
        #print(data)
        pred,_ = model(data)
        pred = pred.max(1,keepdim=True)[1].type(torch.Tensor)
        torchvision.utils.save_image(pred,pred_name)
      
      frame = cv2.resize(frame,(256,128))
      cv2.imwrite(name, frame)
      currentframe +=1
  else:
    break

cap.release()
cv2.destroyAllWindows()



creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame1.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame2.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame3.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame4.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame5.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame6.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame7.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame8.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame9.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame10.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame11.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame12.jpg
creating.../content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame13

# cnn

In [ ]:
import cv2 
import os
from torchvision import transforms
import torchvision
convert_tensor = transforms.ToTensor()
video_path = '/content/drive/MyDrive/Video Processing/VID_20220323_170619.mp4'
cap = cv2.VideoCapture(video_path)
try: 
  if not os.path.exists('/content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet'):
    os.makedirs('/content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet')
except OSError: 
  print('Error: Creating directory of data')

currentframe = 0
while(True):
  success,frame = cap.read()
  if success:
    path = '/content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/'
    name = path + 'frame' + str(currentframe) +'.jpg'
    #pred_name = path + 'pred' + str(currentframe) + '.jpg'
    print('creating...'+name)

    with torch.no_grad():
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame_resized = cv2.resize(frame,(256,128))
      frame = cv2.resize(frame,(256,128))
      cv2.imwrite(name, frame)
      frame = convert_tensor(frame).unsqueeze(0).to(device)
      frame = model(frame).type(torch.Tensor).to(device)
      frame = frame.max(1,keepdim=True)[1].type(torch.FloatTensor)
      torchvision.utils.save_image(frame,name)
      #frame = frame.numpy()

    
      #frame = cv2.imread(name)
      #frame[:,:,1] = 0
      #frame[:,:,0] = 0
      #added_image = cv2.addWeighted(frame_resized,1,frame,1,1)
      #combined = cv2.imwrite(name, added_image)
    currentframe +=1
  else:
    break

cap.release()
cv2.destroyAllWindows()



creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame0.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame1.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame2.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame3.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame4.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame5.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame6.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame7.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame8.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet/frame9.jpg
creating.../content/drive/MyDrive/Video Processing/outVID_20220323_170619_SegNet

# Making Video

In [ ]:
import numpy as np
import glob
import cv2
 
img_array = []
for filename in glob.glob('/content/drive/MyDrive/Video Processing/pred3/*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('pred3_mask.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [ ]:
!pip install natsort

In [ ]:
import natsort

In [ ]:
from natsort import os_sorted

# Gabungin frame ke hasil prediksi

In [ ]:
import os
def getListNatsort(dir):
    lst1 = os.listdir(dir)
    lst1 = os_sorted(lst1) 
    i = 0
    lst_dir = []
    for i in range(len(lst1)):
        filename = dir + lst1[i]
        lst_dir.append(filename)
    return lst_dir


In [ ]:
path_image = '/content/drive/MyDrive/Video Processing/cerahlurusTA_2/'
path_pred = '/content/drive/MyDrive/Video Processing/cerahlurusTA_2_UNetLSTM/'
path_result = '/content/drive/MyDrive/Video Processing/cerahlurusTA_2_UNetLSTM_result/'
list_image = getListNatsort(path_image)
list_pred = getListNatsort(path_pred)
len(list_image)
list_image[9]
#list_pred[0]

'/content/drive/MyDrive/Video Processing/cerahlurusTA_2/frame10.jpg'

# Overlay Image every 6 images:
#### --> : overlay
### list_image[0] --> list_pred[0]
### list_image[1] --> list_pred[0]
### list_image[2] --> list_pred[0]
### list_image[3] --> list_pred[0]
### list_image[4] --> list_pred[0]
### list_image[5] --> list_pred[0]
### list_image[6] --> list_pred[1]
### list_image[7] --> list_pred[1]

In [ ]:
import cv2
current_pred = 0 

for i in range(len(ordered)+1):
  if i == 0:
    name = path_result + 'frame' + str(i) +'.jpg'
    img = cv2.imread(ordered[i])
    mask = cv2.imread(ordered_pred[current_pred])
    mask[:,:,1] = 0
    mask[:,:,0] = 0
    added_image = cv2.addWeighted(img,1,mask,0.5,1)
    combined = cv2.imwrite(name,added_image)
  if i % 6 == 0 and i != 0  :
    img = cv2.imread(ordered[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(ordered_pred[current_pred])
    mask[:,:,1] = 0
    mask[:,:,0] = 0
    added_image = cv2.addWeighted(img,1,mask,1,1)
    combined = cv2.imwrite(name,added_image)
    current_pred +=1
  name = path_result + 'frame' + str(i) +'.jpg'
  print(name)
  img = cv2.imread(ordered[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  if i == 2888:
    break
  mask = cv2.imread(ordered_pred[current_pred])
  mask[:,:,1] = 0
  mask[:,:,0] = 0
  added_image = cv2.addWeighted(img,1,mask,0.5,1)
  combined = cv2.imwrite(name,added_image)
  

/content/drive/MyDrive/Video Processing/overlay_pred3/frame0.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame1.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame2.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame3.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame4.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame5.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame6.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame7.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame8.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame9.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame10.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame11.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame12.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame13.jpg
/content/drive/MyDrive/Video Processing/overlay_pred3/frame14.jpg
/content/drive/MyDri

IndexError: ignored

In [ ]:
import cv2
current_mask = 0
#current_frame = 0
for i in range(5, (len(list_image)+1)):
  if i in range((len(list_image)-5), (len(list_image))):
    name = path_result + 'frame' + str(i) +'.jpg'
    img = cv2.imread(list_image[i-1])
    mask = cv2.imread(list_pred[current_mask-1])
    mask[:,:,1] = 0
    mask[:,:,0] = 0
    added_image = cv2.addWeighted(img,1,mask,0.5,1)
    combined = cv2.imwrite(name,added_image)
  if i % 5 == 0:
    current_mask += 1
  

  name = path_result + 'frame' + str(i) +'.jpg'
  img = cv2.imread(list_image[i-1])
  mask = cv2.imread(list_pred[current_mask])
  mask[:,:,1] = 0
  mask[:,:,0] = 0
  added_image = cv2.addWeighted(img,1,mask,0.5,1)
  combined = cv2.imwrite(name,added_image)


IndexError: ignored

In [ ]:
range((len(list_image)-4),len(list_image))

range(550, 554)

# Make Video from Frames

In [ ]:
import numpy as np
import cv2
 
img_array = []
for filename in getListNatsort('/content/drive/MyDrive/Video Processing/cerahlurusTA_2_UNetLSTM_result/'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('cerahlurusUNetLSTMResult.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
